<a href="https://colab.research.google.com/github/ShrikantKGIT/general/blob/main/SimPL_A_Custom_Language%2C_Compiler%2C_and_Interpreter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Complete system for a custom programming language named "SimPL".**

The Python script is divided into three main parts:

**SimPLInterpreter**: This is the virtual machine. It's a simple, stack-based machine that knows how to perform fundamental operations like loading constants, adding numbers, and storing variables. It only understands the low-level "bytecode."

**SimPLCompiler**: This is the translator. It takes the human-readable SimPL source code and converts each line into a series of simple bytecode instructions that the interpreter can execute.

**Example Usage**: The main block shows the entire process. A sample program in SimPL, which is then compiled into bytecode, and finally, that bytecode is executed by the interpreter to produce the final result.

In [1]:
import re

# Part 1: The Interpreter (Virtual Machine)
# We define the interpreter first because the compiler needs to know what
# instructions it can generate.

class SimPLInterpreter:
    """
    Executes the bytecode generated by the SimPLCompiler.
    This is a stack-based virtual machine.
    """
    def __init__(self):
        self.stack = []
        self.environment = {}

    def run_code(self, instructions):
        """
        Executes a list of bytecode instructions.

        Args:
            instructions (list): A list of instruction tuples.
        """
        print("--- Running Bytecode ---")
        for command, *args in instructions:
            argument = args[0] if args else None

            # Dispatch to the correct method
            if hasattr(self, f"op_{command}"):
                getattr(self, f"op_{command}")(argument)
            else:
                print(f"Error: Unknown bytecode instruction '{command}'")
                return
        print("--- Execution Finished ---\n")

    # --- Bytecode Operation Implementations ---

    def op_LOAD_CONST(self, value):
        """Pushes a constant value onto the stack."""
        self.stack.append(value)

    def op_LOAD_VAR(self, name):
        """Retrieves a variable's value and pushes it onto the stack."""
        if name not in self.environment:
            raise NameError(f"Variable '{name}' not defined.")
        self.stack.append(self.environment[name])

    def op_STORE_VAR(self, name):
        """Pops a value from the stack and stores it in a variable."""
        if not self.stack:
            raise IndexError("STORE_VAR requires a value on the stack.")
        self.environment[name] = self.stack.pop()

    def op_ADD(self, _):
        """Pops two values, adds them, and pushes the result."""
        right = self.stack.pop()
        left = self.stack.pop()
        self.stack.append(left + right)

    def op_SUB(self, _):
        """Pops two values, subtracts them, and pushes the result."""
        right = self.stack.pop()
        left = self.stack.pop()
        self.stack.append(left - right)

    def op_PRINT(self, _):
        """Pops a value from the stack and prints it."""
        print(f"Output: {self.stack.pop()}")


# Part 2: The Compiler
# This translates the high-level SimPL language into low-level bytecode.

class SimPLCompiler:
    """
    Compiles SimPL source code into bytecode for the interpreter.
    """
    def compile(self, code):
        """
        Takes a string of SimPL code and returns a list of bytecode instructions.
        """
        bytecode = []
        lines = code.strip().split('\n')

        print("--- Compiling SimPL Code ---")
        for line in lines:
            line = line.strip()
            if not line or line.startswith('#'):
                continue

            print(f"Compiling line: '{line}'")
            parts = re.split(r'[ ,]+', line, maxsplit=2)
            command = parts[0].upper()
            args = parts[1:]

            if command == 'SET':
                # SET var, value_or_var
                var_name = args[0]
                value_str = args[1]
                # Check if the value is a number or another variable
                if value_str.isdigit() or (value_str.startswith('-') and value_str[1:].isdigit()):
                    bytecode.append(('LOAD_CONST', int(value_str)))
                else:
                    bytecode.append(('LOAD_VAR', value_str))
                bytecode.append(('STORE_VAR', var_name))

            elif command in ('ADD', 'SUB'):
                # ADD dest_var, source_val_or_var
                dest_var = args[0]
                source_str = args[1]
                # First, load the value to add/subtract
                if source_str.isdigit() or (source_str.startswith('-') and source_str[1:].isdigit()):
                    bytecode.append(('LOAD_CONST', int(source_str)))
                else:
                    bytecode.append(('LOAD_VAR', source_str))
                # Next, load the destination variable's current value
                bytecode.append(('LOAD_VAR', dest_var))
                # Perform the operation
                bytecode.append((command, None))
                # Store the result back in the destination variable
                bytecode.append(('STORE_VAR', dest_var))

            elif command == 'PRINT':
                # PRINT var
                var_name = args[0]
                bytecode.append(('LOAD_VAR', var_name))
                bytecode.append(('PRINT', None))

            else:
                raise SyntaxError(f"Unknown command: {command}")

        print("--- Compilation Finished ---\n")
        return bytecode


# Part 3: Example Usage

if __name__ == "__main__":
    # This is our program written in the custom SimPL language.
    simpl_program = """
    # Simple calculator program in SimPL
    # It calculates result = (100 - 50) + (x - 5) where x is 20

    SET x, 20
    SET y, 100

    # Perform y = y - 50
    SUB y, 50

    # Perform x = x - 5
    SUB x, 5

    # Perform result = y + x
    SET result, y
    ADD result, x

    PRINT result
    """

    print("--- Source Code in SimPL ---")
    print(simpl_program)
    print("----------------------------\n")

    # 1. Instantiate the compiler
    compiler = SimPLCompiler()

    # 2. Compile the source code into bytecode
    program_bytecode = compiler.compile(simpl_program)

    print("--- Generated Bytecode ---")
    import pprint
    pprint.pprint(program_bytecode)
    print("--------------------------\n")

    # 3. Instantiate the interpreter
    interpreter = SimPLInterpreter()

    # 4. Run the compiled bytecode
    interpreter.run_code(program_bytecode)

    # You can inspect the final state of the interpreter
    print("--- Final Interpreter State ---")
    print("Environment:", interpreter.environment)
    print("Stack:", interpreter.stack)


--- Source Code in SimPL ---

    # Simple calculator program in SimPL
    # It calculates result = (100 - 50) + (x - 5) where x is 20

    SET x, 20
    SET y, 100

    # Perform y = y - 50
    SUB y, 50

    # Perform x = x - 5
    SUB x, 5

    # Perform result = y + x
    SET result, y
    ADD result, x

    PRINT result
    
----------------------------

--- Compiling SimPL Code ---
Compiling line: 'SET x, 20'
Compiling line: 'SET y, 100'
Compiling line: 'SUB y, 50'
Compiling line: 'SUB x, 5'
Compiling line: 'SET result, y'
Compiling line: 'ADD result, x'
Compiling line: 'PRINT result'
--- Compilation Finished ---

--- Generated Bytecode ---
[('LOAD_CONST', 20),
 ('STORE_VAR', 'x'),
 ('LOAD_CONST', 100),
 ('STORE_VAR', 'y'),
 ('LOAD_CONST', 50),
 ('LOAD_VAR', 'y'),
 ('SUB', None),
 ('STORE_VAR', 'y'),
 ('LOAD_CONST', 5),
 ('LOAD_VAR', 'x'),
 ('SUB', None),
 ('STORE_VAR', 'x'),
 ('LOAD_VAR', 'y'),
 ('STORE_VAR', 'result'),
 ('LOAD_VAR', 'x'),
 ('LOAD_VAR', 'result'),
 ('ADD', None)